###### Importing all the required packages as a first step.

In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
from PIL import  Image
%matplotlib inline
import matplotlib.ticker as mtick
import seaborn as sns
sns.set(style="ticks", color_codes=True)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
sns.set(style="white")
import io

##### Importing data files

In [2]:
prop2016 = pd.read_csv("Raw data/properties_2016.csv", low_memory=False)
prop2017 = pd.read_csv("Raw data/properties_2017.csv", low_memory=False)
train2016 = pd.read_csv("Raw data/train_2016_v2.csv", low_memory=False, parse_dates=['transactiondate'])
train2017 = pd.read_csv("Raw data/train_2017.csv", low_memory=False, parse_dates=['transactiondate'])
print(prop2016.shape)
print(prop2017.shape)
print(train2016.shape)
print(train2017.shape)

(2985217, 58)
(2985217, 58)
(90275, 3)
(77613, 3)


###### Checking the top 5 rows to make sure the data is loaded fine

In [3]:
print(prop2016.head())
print(train2016.head())

   parcelid  airconditioningtypeid  architecturalstyletypeid  basementsqft  \
0  10754147                    NaN                       NaN           NaN   
1  10759547                    NaN                       NaN           NaN   
2  10843547                    NaN                       NaN           NaN   
3  10859147                    NaN                       NaN           NaN   
4  10879947                    NaN                       NaN           NaN   

   bathroomcnt  bedroomcnt  buildingclasstypeid  buildingqualitytypeid  \
0          0.0         0.0                  NaN                    NaN   
1          0.0         0.0                  NaN                    NaN   
2          0.0         0.0                  NaN                    NaN   
3          0.0         0.0                  3.0                    7.0   
4          0.0         0.0                  4.0                    NaN   

   calculatedbathnbr  decktypeid  ...  numberofstories  fireplaceflag  \
0            

As we can see from the above outputs, 'train2016' dataset has all the transactions done in 2016 and 'prop2016' has the property features in 2016. similarly 'train2017' and 'prop2017' contains data of 2017.

Transactions data should be merged with properties data to get the dataset with predictor and target variables.

In [4]:
trainingdata2016 = train2016.merge(prop2016,on=['parcelid'])
trainingdata2016.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [5]:
trainingdata2017 = train2017.merge(prop2017,on=['parcelid'])

#### Next steps are to concat both 2016 & 2017 datasets and understand the size of the dataset  

In [6]:
#fulldata=trainingdata2016.append(trainingdata2017,ignore_index=True)
fulldata=pd.concat([trainingdata2016,trainingdata2017],axis=0,join="outer")
fulldata.shape

(167888, 60)

#### Splitting data into train and test

In [7]:
print(datetime.date(2017,10,15))
print(pd.Timestamp(2017,10,15))

2017-10-15
2017-10-15 00:00:00


In [8]:
fulldata_test = fulldata[(fulldata['transactiondate']>=pd.Timestamp(2017,10,15))]
fulldata=fulldata[(fulldata['transactiondate'] < pd.Timestamp(2017,10,15))]
print(fulldata.shape)
print(fulldata_test.shape)

(167888, 60)
(0, 60)


In [9]:
#fulldata.info()
datatypes=fulldata.dtypes.reset_index()
datatypes.columns=["column","columntype"]
datatypes

,column,columntype
0,parcelid,int64
1,logerror,float64
2,transactiondate,datetime64[ns]
3,airconditioningtypeid,float64
4,architecturalstyletypeid,float64
5,basementsqft,float64
6,bathroomcnt,float64
7,bedroomcnt,float64
8,buildingclasstypeid,float64
9,buildingqualitytypeid,float64


In [10]:
#Getting the count of columns by datatype
datatypes.groupby("columntype").count().reset_index()

,columntype,column
0,int64,1
1,float64,53
2,datetime64[ns],1
3,object,5


#### checking for missing data and converting all variables to the right datatypes

In [11]:
missingdata=fulldata.isnull().sum()/len(fulldata)
missingdata.sort_values(ascending=False)

buildingclasstypeid             0.999815
finishedsquarefeet13            0.999553
basementsqft                    0.999446
storytypeid                     0.999446
yardbuildingsqft26              0.999017
fireplaceflag                   0.997653
architecturalstyletypeid        0.997212
typeconstructiontypeid          0.996891
finishedsquarefeet6             0.995193
decktypeid                      0.992424
pooltypeid10                    0.990315
poolsizesum                     0.989052
pooltypeid2                     0.986431
hashottuborspa                  0.976746
taxdelinquencyyear              0.972106
taxdelinquencyflag              0.972106
yardbuildingsqft17              0.969986
finishedsquarefeet15            0.960742
finishedsquarefeet50            0.923205
finishedfloor1squarefeet        0.923205
fireplacecnt                    0.893405
threequarterbathnbr             0.868275
pooltypeid7                     0.810731
poolcnt                         0.797037
numberofstories 

##### Droping columns for which atleast 90% of the values are missing

In [12]:
#Identifying columns with only one value
def uniq_val(dataset):
    cols = []
    for i in dataset.columns.values:
        if(len(dataset[i].unique())<=2):
            cols.append(i)
    return cols
uniq_val(fulldata)

['decktypeid',
 'hashottuborspa',
 'poolcnt',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'storytypeid',
 'fireplaceflag',
 'taxdelinquencyflag']

In [13]:
#fulldata.buildingclasstypeid.unique()
#fulldata.groupby('buildingclasstypeid')['parcelid'].count()
print(fulldata.decktypeid.unique())
print(fulldata.hashottuborspa.unique())
print(fulldata.poolcnt.unique())
print(fulldata.pooltypeid10.unique())
print(fulldata.pooltypeid2.unique())
print(fulldata.pooltypeid7.unique())
print(fulldata.storytypeid.unique())
print(fulldata.fireplaceflag.unique())
print(fulldata.taxdelinquencyflag.unique())

[nan 66.]
[nan True]
[nan  1.]
[nan  1.]
[nan  1.]
[nan  1.]
[nan  7.]
[nan True]
[nan 'Y']


In [14]:
fulldata['hashottuborspa']=fulldata['hashottuborspa'].replace(np.nan,0)
fulldata['hashottuborspa']=fulldata['hashottuborspa'].replace(True,1)
print(fulldata.hashottuborspa.unique())
fulldata['poolcnt']=fulldata['poolcnt'].replace(np.nan,0)
print(fulldata.poolcnt.unique())
fulldata['pooltypeid10']=fulldata['pooltypeid10'].replace(np.nan,0)
print(fulldata.pooltypeid10.unique())
fulldata['pooltypeid2']=fulldata['pooltypeid2'].replace(np.nan,0)
print(fulldata.pooltypeid2.unique())
fulldata['pooltypeid7']=fulldata['pooltypeid7'].replace(np.nan,0)
print(fulldata.pooltypeid7.unique())
fulldata['fireplaceflag']=fulldata['fireplaceflag'].replace(np.nan,0)
fulldata['fireplaceflag']=fulldata['fireplaceflag'].replace(True,1)
print(fulldata.fireplaceflag.unique())
fulldata['taxdelinquencyflag']=fulldata['taxdelinquencyflag'].replace(np.nan,0)
fulldata['taxdelinquencyflag']=fulldata['taxdelinquencyflag'].replace('Y',1)
print(fulldata.taxdelinquencyflag.unique())

[0 1]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0 1]
[0 1]


In [15]:
#Identifying columns with 90% missing data
missing_df = fulldata.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df['missing_ratio'] = missing_df['missing_count'] / fulldata.shape[0]
m=missing_df.ix[missing_df['missing_ratio']>0.90]
m.sort_values('missing_ratio', ascending=False)
print(m['column_name'].values)

#Dropping columns with 90% missing data
fulldata.drop(m['column_name'].values,axis=1,inplace=True)

['architecturalstyletypeid' 'basementsqft' 'buildingclasstypeid'
 'decktypeid' 'finishedfloor1squarefeet' 'finishedsquarefeet13'
 'finishedsquarefeet15' 'finishedsquarefeet50' 'finishedsquarefeet6'
 'poolsizesum' 'storytypeid' 'typeconstructiontypeid' 'yardbuildingsqft17'
 'yardbuildingsqft26' 'taxdelinquencyyear']


##### Imputing missing data with median for non-categorical variables

In [16]:
fulldata.iloc[:,0:46].head()

,parcelid,logerror,transactiondate,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,2.0,3.0,4.0,2.0,1684.0,1684.0,...,1959.0,NaN,0,122754.0,360170.0,2015.0,237416.0,6735.88,0,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,3.5,4.0,NaN,3.5,2263.0,2263.0,...,2014.0,NaN,0,346458.0,585529.0,2015.0,239071.0,10153.02,0,NaN
2,12098116,-0.0040,2016-01-01,1.0,3.0,2.0,4.0,3.0,2217.0,2217.0,...,1940.0,NaN,0,61994.0,119906.0,2015.0,57912.0,11484.48,0,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,2.0,2.0,4.0,2.0,839.0,839.0,...,1987.0,NaN,0,171518.0,244880.0,2015.0,73362.0,3048.74,0,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,2.5,4.0,NaN,2.5,2283.0,2283.0,...,1981.0,2.0,0,169574.0,434551.0,2015.0,264977.0,5488.96,0,6.059042e+13


In [17]:
#Going throught the data dictionary variables we get the list of variables that should be continuous numeric,
# discrete numeric and categorical
dn_vars = ['airconditioningtypeid','bathroomcnt','bedroomcnt','buildingqualitytypeid','calculatedbathnbr',
           'fireplacecnt','fullbathcnt','garagecarcnt','heatingorsystemtypeid','roomcnt','threequarterbathnbr','unitcnt',
          'yearbuilt','numberofstories','assessmentyear']
cn_vars = ['calculatedfinishedsquarefeet','finishedsquarefeet12','fips','garagetotalsqft','latitude','longitude',
           'lotsizesquarefeet','structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt','taxamount']
cat_vars = ['propertycountylandusecode','propertylandusetypeid','propertyzoningdesc','rawcensustractandblock',
            'regionidcity','regionidcounty','regionidneighborhood','regionidzip','censustractandblock']

In [18]:
# replacing missing values of discrete numeric variables with median
for i in dn_vars:
    fulldata[i]=fulldata[i].replace(np.nan,np.nanmedian(fulldata[i]))

In [19]:
# replacing missing values of continuous numeric variables with mean
for i in cn_vars:
    fulldata[i]=fulldata[i].replace(np.nan,np.mean(fulldata[i]))

In [20]:
# replacing missing values of categorical variables with special category
for i in cat_vars:
    fulldata[i]=fulldata[i].replace(np.nan,9999999999)

In [21]:
# Checking for missing data again to see if the imputation was performed fully
missingdata=fulldata.isnull().sum()/len(fulldata)
missingdata.sort_values(ascending=False)

censustractandblock             0.0
pooltypeid10                    0.0
lotsizesquarefeet               0.0
longitude                       0.0
latitude                        0.0
heatingorsystemtypeid           0.0
hashottuborspa                  0.0
garagetotalsqft                 0.0
garagecarcnt                    0.0
fullbathcnt                     0.0
fireplacecnt                    0.0
fips                            0.0
finishedsquarefeet12            0.0
calculatedfinishedsquarefeet    0.0
calculatedbathnbr               0.0
buildingqualitytypeid           0.0
bedroomcnt                      0.0
bathroomcnt                     0.0
airconditioningtypeid           0.0
transactiondate                 0.0
logerror                        0.0
poolcnt                         0.0
pooltypeid2                     0.0
taxdelinquencyflag              0.0
pooltypeid7                     0.0
taxamount                       0.0
landtaxvaluedollarcnt           0.0
assessmentyear              

In [22]:
for col in cat_vars:
    fulldata[col]=fulldata[col].astype("category")

#### Checking correlation between variables and deleting variables that have greater than 0.9 correlation

In [23]:
X = fulldata.drop(['parcelid','transactiondate','logerror'],axis=1)
Y = fulldata.logerror
corr_matrix=X.corr()

In [24]:
X1 = X.copy()
print(X1.shape)
del_col=[]
for i in range(corr_matrix.shape[0]):
    for j in range(i):
        if(abs(corr_matrix.iloc[i,j])>=0.9):
            if(j not in del_col):
                X1.drop(corr_matrix.columns[i],axis=1,inplace=True)
            del_col.append(i)
X1.shape

(167888, 42)


(167888, 36)

In [31]:
#Checking for the number of unique values in each categorical variable
for col in cat_vars:
    print(col)
    print(len(fulldata[col].unique()))

propertycountylandusecode
91
propertylandusetypeid
15
propertyzoningdesc
2347
rawcensustractandblock
57812
regionidcity
179
regionidcounty
4
regionidneighborhood
506
regionidzip
391
censustractandblock
57545


In [59]:
p=fulldata.propertycountylandusecode.value_counts().reset_index()
p.columns=["value","cnt"]
print(p.shape)
print(p[p.cnt<1000].shape)
p

(91, 2)
(77, 2)


,value,cnt
0,0100,57628
1,122,28450
2,010C,19102
3,0101,13848
4,34,10878
...,...,...
86,01DD,1
87,010L,1
88,1200,1
89,0133,1


In [89]:
# Clubbing all the values that seem to be 'non-freq'
freq_val=p.loc[p.cnt>1000,'value']
print(freq_val)
fd=fulldata['propertycountylandusecode'].replace(dict.fromkeys([i for i in fulldata['propertycountylandusecode']
                                                                if i not in freq_val],'999999'))
fd.value_counts()
#fulldata['propertycountylandusecode'] in freq_val

0     0100
1      122
2     010C
3     0101
4       34
5     1111
6        1
7     010E
8     010D
9     0200
10    1129
11    1110
12    0400
13    0300
Name: value, dtype: category
Categories (91, object): [9999999999, 0, 010, 0100, ..., 6050, 73, 8800, 96]


999999    167888
Name: propertycountylandusecode, dtype: int64

In [32]:
# Dropping 'censustractandblock', 'rawcensustractandblock' & 'propertyzoningdesc' as they have too many unique values 
fulldata.drop(['censustractandblock','rawcensustractandblock','propertyzoningdesc'],axis=1,inplace=True)

### Model Building

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X1, Y, test_size=0.3)

In [27]:
rfc=RandomForestClassifier()
model=rfc.fit(X_train,y_train)
prediction_rfc=rfc.predict(X_test)
print(metrics.accuracy_score(y_test, prediction_rfc))

ValueError: could not convert string to float: '010C'